#  Consultas SPARQL: Análisis del Grafo de Agricultura de Precisión

Este notebook contiene **15 consultas SPARQL** diseñadas para extraer información del grafo RDF generado previamente (`out/agri_graph.ttl`).

Las consultas cubren:
*   Consultas básicas (SELECT)
*   Filtrado (FILTER, REGEX)
*   Agrupamiento y Agregación (GROUP BY, COUNT, ORDER BY)
*   Property Paths (Búsqueda en profundidad)
*   Subqueries y Lógica condicional


In [1]:
import pandas as pd
from rdflib import Graph

# Cargar el grafo generado
g = Graph()
g.parse("out/agri_graph.ttl", format="turtle")

print(f"✅ Grafo cargado con {len(g)} triples.")

# Función helper para mostrar resultados en DataFrame
def query_to_df(query):
    results = g.query(query)
    data = []
    for row in results:
        data.append({str(k): str(v) for k, v in row.asdict().items()})
    return pd.DataFrame(data)


✅ Grafo cargado con 69174 triples.


### 1. Listar 10 artículos (URI y Título)
Consulta básica para recuperar identificadores y títulos de artículos.

In [2]:
query = """
PREFIX schema: <http://schema.org/>
SELECT ?art ?title
WHERE {
  ?art a schema:Article ;
       schema:title ?title .
}
LIMIT 10
"""

df = query_to_df(query)
display(df)


,art,title
0,http://example.org/agri#art-000f4079fb3f5463ce...,The combine will tell the truth: On precision ...
1,http://example.org/agri#art-0067d4c3e173138fe9...,Determination of chemical soil properties usin...
2,http://example.org/agri#art-00b72b92171de69c7d...,Determination of chemical soil properties usin...
3,http://example.org/agri#art-00e65f396b4febdc79...,Achlys: Towards a Framework for Distributed St...
4,http://example.org/agri#art-01032be0c159e67035...,Comm4Dev: Communication Infrastructure for Dev...
5,http://example.org/agri#art-01623a7875aeb1eb89...,Rhocop: Receding Horizon Multi-Robot Coverage
6,http://example.org/agri#art-018cd553400718f0d1...,Exploring the impact of innovation adoption in...
7,http://example.org/agri#art-01b3ec2ae24e72bfc4...,A hybrid classifier approach to multivariate s...
8,http://example.org/agri#art-01d3325a5544e4b7e6...,Enhancement Of Segmentation And Feature Fusion...
9,http://example.org/agri#art-02319ec3fafbe141df...,Preface


### 2. Autores con 'Smith' en su nombre
Uso de `FILTER` y `REGEX` para búsqueda de texto.

In [3]:
query = """
PREFIX schema: <http://schema.org/>
SELECT ?person ?name
WHERE {
  ?person a schema:Person ;
          schema:name ?name .
  FILTER REGEX(?name, "Smith", "i")
}
LIMIT 10
"""

df = query_to_df(query)
display(df)


""


### 3. Conteo total de artículos y autores
Uso de `UNION` o conteos separados (aquí usamos UNION implícito al contar tipos distintos en columnas opcionales, o simplemente dos counts). Haremos un conteo de artículos.

In [4]:
query = """
PREFIX schema: <http://schema.org/>
SELECT (COUNT(?art) AS ?totalArticles)
WHERE {
  ?art a schema:Article .
}
"""

df = query_to_df(query)
display(df)


,totalArticles
0,13747


### 4. Cantidad de artículos por año
Uso de `GROUP BY` y `ORDER BY`.

In [5]:
query = """
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?year (COUNT(?art) AS ?count)
WHERE {
  ?art a schema:Article ;
       schema:datePublished ?date .
  BIND(YEAR(?date) AS ?year)
}
GROUP BY ?year
ORDER BY DESC(?year)
"""

df = query_to_df(query)
display(df)


,count
0,1000


### 5. Top 10 Venues (Revistas/Conferencias) con más publicaciones
Agrupamiento por entidad relacionada (`schema:isPartOf`).

In [6]:
query = """
PREFIX schema: <http://schema.org/>
SELECT ?venueName (COUNT(?art) AS ?count)
WHERE {
  ?art a schema:Article ;
       schema:isPartOf ?venue .
  ?venue schema:name ?venueName .
}
GROUP BY ?venueName
ORDER BY DESC(?count)
LIMIT 10
"""

df = query_to_df(query)
display(df)


,venueName,count
0,Remote Sensing,42
1,Italian National Conference on Sensors,34
2,IOP Conference Series: Earth and Environment,22
3,Precision Agriculture,19
4,IEEE International Geoscience and Remote Sensi...,18
5,IEEE Access,16
6,International Conference on Agro-Geoinformatics,14
7,Journal of Physics: Conference Series,10
8,IOP Conference Series: Materials Science and E...,9
9,arXiv.org,9


### 6. Top 10 Conceptos (Topics) más frecuentes
Análisis de temáticas usando `dct:subject`.

In [7]:
query = """
PREFIX schema: <http://schema.org/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?conceptLabel (COUNT(?art) AS ?count)
WHERE {
  ?art dct:subject ?concept .
  ?concept skos:prefLabel ?conceptLabel .
}
GROUP BY ?conceptLabel
ORDER BY DESC(?count)
LIMIT 10
"""

df = query_to_df(query)
display(df)


,conceptLabel,count
0,Computer Science,631
1,Medicine,169
2,Environmental Science,145
3,Business,67
4,Biology,51
5,Physics,48
6,Engineering,41
7,Mathematics,40
8,Geology,34
9,Chemistry,31


### 7. Artículos sobre un tema específico (ej. 'Machine learning')
Filtrado por concepto específico.

In [8]:
query = """
PREFIX schema: <http://schema.org/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?title
WHERE {
  ?art a schema:Article ;
       schema:title ?title ;
       dct:subject ?concept .
  ?concept skos:prefLabel ?label .
  FILTER (LCASE(?label) = "machine learning")
}
LIMIT 10
"""

df = query_to_df(query)
display(df)


""


### 8. Autores que han publicado sobre 'Remote sensing'
Navegación de grafo: `Person` <- `author` - `Article` - `subject` -> `Concept`.

In [9]:
query = """
PREFIX schema: <http://schema.org/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?authorName
WHERE {
  ?art dct:subject ?concept ;
       schema:author ?person .
  ?concept skos:prefLabel ?label .
  ?person schema:name ?authorName .
  FILTER (REGEX(?label, "Remote sensing", "i"))
}
LIMIT 15
"""

df = query_to_df(query)
display(df)


""


### 9. Pares de co-autores (escribieron juntos)
Self-join: Dos autores distintos que comparten el mismo artículo.

In [10]:
query = """
PREFIX schema: <http://schema.org/>

SELECT ?author1 ?author2 (COUNT(?art) as ?collabCount)
WHERE {
  ?art schema:author ?p1 .
  ?art schema:author ?p2 .
  ?p1 schema:name ?author1 .
  ?p2 schema:name ?author2 .
  FILTER (STR(?p1) < STR(?p2)) # Evitar duplicados simétricos (A-B y B-A)
}
GROUP BY ?author1 ?author2
ORDER BY DESC(?collabCount)
LIMIT 10
"""

df = query_to_df(query)
display(df)


,author1,author2,collabCount
0,A. Mancini,P. Zingaretti,5
1,D. Popescu,L. Ichim,5
2,A. Mancini,E. Frontoni,4
3,P. Zingaretti,E. Frontoni,4
4,Jiancheng Luo,Yingwei Sun,4
5,C. Stachniss,Nived Chebrolu,4
6,C. Stachniss,Philipp Lottes,4
7,M. Belletti,D. Bentivoglio,3
8,Robison Cris Brito,E. Todt,3
9,W. Cao,Yan Zhu,3


### 10. Artículos que citan a artículos populares (>10 citas)
Uso de property paths para citas (`schema:citation`) y filtrado por métrica (asumiendo que cargamos `ex:citationCount`).

In [11]:
query = """
PREFIX schema: <http://schema.org/>
PREFIX ex: <http://example.org/agri#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?sourceTitle ?citedTitle ?citedCount
WHERE {
  ?source a schema:Article ;
          schema:title ?sourceTitle ;
          schema:citation ?target .
  
  ?target schema:title ?citedTitle ;
          ex:citationCount ?citedCount .
          
  FILTER (?citedCount > 50)
}
LIMIT 10
"""

df = query_to_df(query)
display(df)


,sourceTitle,citedTitle,citedCount
0,Exploring the impact of innovation adoption in...,Influencing factors and incentives on the inte...,105
1,Agricultural Robotics,Crop Row Detection on Tiny Plants With the Pat...,88
2,Agricultural Robotics,Hyperspectral Sensors and Imaging Technologies...,252
3,Deep Learning for Soil and Crop Segmentation f...,WeedMap: A large-scale semantic weed mapping f...,238
4,An Approach of Beans Plant Development Classif...,Computer vision and artificial intelligence in...,734
5,CropDeep: The Crop Vision Dataset for Deep-Lea...,Computer vision and artificial intelligence in...,734
6,Spectral Data Collection by Dual Field-of-View...,Leaf and Canopy Level Detection of Fusarium Vi...,52
7,Dynamic Physiological Phenotyping of Drought-S...,High-Throughput Plant Phenotyping for Developi...,149
8,Unmanned Aircraft System (UAS) Technology and ...,AgriColMap: Aerial-Ground Collaborative 3D Map...,74
9,Unmanned Aircraft System (UAS) Technology and ...,Research and development in agricultural robot...,455


### 11. Artículos sin DOI
Uso de `FILTER NOT EXISTS` o `OPTIONAL` + `!BOUND` para encontrar datos faltantes.

In [12]:
query = """
PREFIX schema: <http://schema.org/>

SELECT ?title
WHERE {
  ?art a schema:Article ;
       schema:title ?title .
  FILTER NOT EXISTS { ?art schema:doi ?doi }
}
LIMIT 10
"""

df = query_to_df(query)
display(df)


,title
0,Crop plant reconstruction and feature extracti...
1,Assessing a mobile-based deep learning model f...
2,New Breeding Techniques and Their Possible Reg...
3,Introducing innovative precision farming techn...
4,A Tube-based Robust MPC for a Fixed-wing UAV: ...
5,Isolines in 3D Radar Images for Remote Sensing...
6,Advanced Buried Object Detection by Multichann...
7,Toward open smart IoT Systems: an overview of ...
8,Factors Impacting the Design of a Mobile Appli...
9,A Survey Paper on Applications of Soft Computi...


### 12. Promedio de citas de los artículos en el grafo
Agregación global con `AVG`.

In [13]:
query = """
PREFIX schema: <http://schema.org/>
PREFIX ex: <http://example.org/agri#>

SELECT (AVG(?cc) AS ?avgCitations)
WHERE {
  ?art a schema:Article ;
       ex:citationCount ?cc .
}
"""

df = query_to_df(query)
display(df)


,avgCitations
0,38.142


### 13. Autores 'prolíficos' (Más de 5 pappers)
Subquery o Group By con `HAVING`.

In [14]:
query = """
PREFIX schema: <http://schema.org/>

SELECT ?authorName (COUNT(?art) AS ?paperCount)
WHERE {
  ?art schema:author ?person .
  ?person schema:name ?authorName .
}
GROUP BY ?authorName
HAVING (COUNT(?art) > 5)
ORDER BY DESC(?paperCount)
LIMIT 10
"""

df = query_to_df(query)
display(df)


,authorName,paperCount
0,C. Stachniss,7


### 14. Property Path: Conceptos citados indirectamente
Artículos que tratan un tema y citan a otro artículo que trata sobre 'Deep learning'. (Cadena: Art(T1) -> cites -> Art(DL)).

In [15]:
query = """
PREFIX schema: <http://schema.org/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?topicSource
WHERE {
  ?source a schema:Article ;
          dct:subject/skos:prefLabel ?topicSource ;
          schema:citation ?target .
  
  ?target dct:subject/skos:prefLabel ?topicTarget .
  
  FILTER (LCASE(?topicTarget) = "deep learning")
  FILTER (LCASE(?topicSource) != "deep learning")
}
LIMIT 10
"""

df = query_to_df(query)
display(df)


""


### 15. Artículos publicados en conferencias (IEEE/ACM/Proc)
Filtrado por nombre del venue usando expresiones regulares.

In [16]:
query = """
PREFIX schema: <http://schema.org/>

SELECT ?title ?venueName
WHERE {
  ?art schema:isPartOf ?venue ;
       schema:title ?title .
  ?venue schema:name ?venueName .
  FILTER (REGEX(?venueName, "IEEE|Conference|Proceedings|ACM", "i"))
}
LIMIT 10
"""

df = query_to_df(query)
display(df)


,title,venueName
0,Achlys: Towards a Framework for Distributed St...,2019 IEEE International Conference on Pervasiv...
1,Comm4Dev: Communication Infrastructure for Dev...,International Conference on e-Infrastructure a...
2,Rhocop: Receding Horizon Multi-Robot Coverage,International Conference on Cyber-Physical Sys...
3,Exploring the impact of innovation adoption in...,IOP Conference Series: Earth and Environment
4,Preface,IOP Conference Series: Earth and Environment
5,The Precision Agriculture Based on Wireless Se...,IOP Conference Series: Earth and Environment
6,Data analysis and inference model for automati...,IOP Conference Series: Earth and Environment
7,Field Information Collection Design and Experi...,IOP Conference Series: Earth and Environment
8,Challenges of multi/hyper spectral images in p...,IOP Conference Series: Earth and Environment
9,A review on the use of drones for precision ag...,IOP Conference Series: Earth and Environment
